In [1]:

import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support
sc = StandardScaler()
import os
import pickle
import fnmatch
import os
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score
from collections import defaultdict

In [2]:
##useful functions

def fwd_dates(_dates_list, _key_date):
    # returns a list of dates that are forward from the key_date
    fwd_dates_list = [i for i in _dates_list if i > _key_date]
    return fwd_dates_list

def common_member(a, b): 
      
    a_set = set(a) 
    b_set = set(b) 
      
    # check length  
    if len(a_set.intersection(b_set)) > 0: 
        return(a_set.intersection(b_set))   
    else: 
        return("no common elements") 


def remove_nans(features_tuple, labels, idx=1):
    # not the cleanest but useful
    # function to clean up nans as I seem to use it a lot, so better to have one function
    # combines the features and labels and removes rows with nans across so we dont lose the ordering
    # returns features and labels
    features_df = pd.concat([features_tuple[0], features_tuple[1], features_tuple[2], \
                             features_tuple[3]], axis=1, sort=False)
    labels_only = labels.drop(columns=['ReturnTradedPrice', 'Duration', 'states', 'TradedTime',
                                       'TradedPrice'], axis=1)
    df_concat = pd.concat([features_df, labels_only.iloc[:, 0:idx]], axis=1, sort='False')
    # only using 1st set of labels- but we can re-write this a bit
    df_x_nan = df_concat.dropna()  # dropping all nans
    label_column_loc_ = df_x_nan.shape[1] - 1  # location of labels column in the clean df
    labels_ = df_x_nan.iloc[:, label_column_loc_:label_column_loc_ + 1]  # keep pure labels
    features_ = df_x_nan.drop(df_x_nan.columns[label_column_loc_], axis=1)  # keeping the features only
    return features_, labels_


def prec_recall_report(y_true, y_predict):
    # function to ge the sci-kit learn classification metrics into a pretty DF for csv!
    report = pd.DataFrame(list(precision_recall_fscore_support(y_true, y_predict)),
                          index=['Precision', 'Recall', 'F1-score', 'Support']).T
    # Now add the 'Avg/Total' row
    report.loc['Avg/Total', :] = precision_recall_fscore_support(y_true, y_predict, average='weighted')
    report.loc['Avg/Total', 'Support'] = report['Support'].sum()
    return report


class DataLoader(object):
    def __init__(self, path_main, ticker):
        self.main_path = path_main
        self.ticker = ticker

        self.features_labels_path = os.path.join(self.main_path, 'features_models')
        self.features_path = os.path.join(self.features_labels_path, 'features')
        # collection of per symbol non directional labels
        self.labels_path = os.path.join(self.features_labels_path, 'labels', self.ticker, 'NON_DIRECTIONAL')
        self.symbol_features_path = os.path.join(self.features_labels_path, 'features', self.ticker, 'MODEL_BASED')
        # list of all the model -oos hmm feature dates - each folder is a collection of oos feature dates
        self.hmm_dates_list = os.listdir(self.symbol_features_path)  # each folder are the OOS features from each HMM
        self.compute_date = os.listdir(os.path.join( \
            self.symbol_features_path, \
            os.listdir(self.symbol_features_path)[1]))[1].split("_")[7]

    def ticker_features(self, model_date, date):
        # need to make this a lot more flexible with number of states
        if model_date < date:
            file_name = "_".join(
                (self.ticker, '3', 'states', 'features', 'date:', date, 'now:', self.compute_date, '.pickle'))
            file_loc = os.path.join(self.symbol_features_path, str(model_date), file_name)
            with open(file_loc, 'rb') as handle:
                ticker_features = pickle.load(handle)
        else:
            print('Loading Feature Date which is in-sample. Change your Model Date')
        return ticker_features

    def ticker_labels_csv(self, date):
        file_loc = os.path.join(self.labels_path, str(date) + '.csv')
        ticker_labels = pd.read_csv(file_loc, index_col=0)
        return ticker_labels

    @staticmethod
    def open_pickle_file(path, pickle_file):
        file_loc = os.path.join(path, pickle_file)
        pickle_to_file = pickle.load(open(file_loc, "rb"))
        return pickle_to_file

    @staticmethod
    def get_date_from_file(file_, numb_):
        return os.path.splitext(file_[numb_])[0]

class MarketFeatures(object):
    # a class to be expanded that uses features for base case -market based only-indicators/features
    """"Requires:
    a dataframe that has TradedPrice And Volume columns
    symbol - A stock symbol on which to form a strategy on.
    short_window - Lookback period for short moving average.
    long_window - Lookback period for long moving average.
    """

    def __init__(self, df):
        #         self.ticker = ticker
        self.df = df

    def load_data(self):
        pass

    def ma_spread(self, short_window=5, long_window=20):
        # function that produces the MA spread, which can be used on its own or as an input for MACD
        short_rolling_px = self.df['TradedPrice'].rolling(window=short_window).mean()
        long_rolling_px = self.df['TradedPrice'].rolling(window=long_window).mean()
        px_name = "_".join(('px_indx', str(short_window), str(long_window)))
        self.df[px_name] = long_rolling_px - short_rolling_px
        return self.df

    def ma_spread_duration(self, short_window=5, long_window=20):
        # function that produces the MA spread, which can be used on its own or as an input for MACD
        short_rolling_px = self.df['Duration'].rolling(window=short_window).mean()
        long_rolling_px = self.df['Duration'].rolling(window=long_window).mean()
        dur_name = "_".join(('dur_indx', str(short_window), str(long_window)))
        self.df[dur_name] = long_rolling_px - short_rolling_px
        return self.df

    def obv_calc(self):
        # on balance volume indicator
        self.df['SignedVolume'] = self.df['Volume'] * np.sign(self.df['TradedPrice'].diff()).cumsum()
        self.df['SignedVolume'].iat[1] = 0
        self.df['OBV'] = self.df['SignedVolume']  # .cumsum()
        self.df = self.df.drop(columns=['SignedVolume'])
        return self.df

    def chaikin_mf(self, period=5):
        # Chaikin money flow indicator
        self.df["MF Multiplier"] = (self.df['TradedPrice'] - (self.df['TradedPrice'].expanding(period).min()) \
                                    - (self.df['TradedPrice'].expanding(period).max() \
                                       - self.df['TradedPrice'])) / (
                                           self.df['TradedPrice'].expanding(period).max() - self.df[ \
                                       'TradedPrice'].expanding(period).min())
        self.df["MF Volume"] = self.df['MF Multiplier'] * self.df['Volume']
        self.df['CMF_' + str(period)] = self.df['MF Volume'].sum() / self.df["Volume"].rolling(period).sum()
        self.df = self.df.drop(columns=['MF Multiplier', 'MF Volume'])
        return self.df

In [3]:
# passport locations

data_oos_loc= ('/media/ak/My Passport/Experiment Data')
passport = ('/media/ak/My Passport')
# experimental_data = os.path.join(passport, 'Experiment Data')
features_loc = os.path.join(data_oos_loc,'features') #
labels_loc = os.path.join(data_oos_loc,'labels')
models_loc = os.path.join(data_oos_loc,'models')
# symbols to use as a starting point
good_symbols = os.listdir(features_loc) 


In [4]:
# pick a symbol that works
symbol ='MKS.L'

syml_features_loc = os.path.join(features_loc,symbol,'MODEL_BASED') #create the symbol feature locations
syml_models_loc = os.path.join(models_loc,symbol,'SINGLE_KERNEL') #create the symbol model locations
syml_labels_loc = os.path.join(labels_loc,symbol,'NON_DIRECTIONAL')# create the symbol labels location


In [5]:
# lists of dates
labels_dates = [os.listdir(syml_labels_loc)[idx].split(".")[0] for idx, _ in enumerate(os.listdir(syml_labels_loc))]
features_dates= os.listdir(syml_features_loc)
model_dates= [os.listdir(syml_models_loc)[idx].split("_")[1] for idx,_ in enumerate(os.listdir(syml_models_loc))]

In [6]:
common_dates=sorted(list(common_member(sorted(list(common_member(features_dates, model_dates))),labels_dates)))


In [7]:
#symbol_compute_date
symbol_compute_date = os.listdir(os.path.join(syml_features_loc,os.listdir(syml_features_loc)[1]))[1].split("_")[7]


In [8]:
#model date- this date corresponds to the model which was used for out of sample
date_idx = 1 
# #contains the list of detailed features for the specific model date
# os.listdir(os.path.join(syml_features_loc, common_dates[date_idx])) 


In [12]:
print symbol
from datetime import datetime
from datetime import timedelta

#create a time-delta which will be needed for the common date
daysdelta=timedelta(days=1)

accuracy_results=defaultdict(dict)
accuracy_models_results=defaultdict(dict)
recall_models_results=defaultdict(dict)
f1_models_results=defaultdict(dict)

# now lets take all the model directories and locations##

for common_date in common_dates:
    #move the common date (which is the hmm date, and folder name) one dat forward
    common_day_start=datetime.strptime(common_date, '%Y%m%d') +daysdelta
    #convert it back to a string (prob we can do in one go)
    first_oos_day= common_day_start.strftime('%Y%m%d')
    # getting the model
    model_name ="_".join((symbol,common_date,'label_PrMov__window_5__thres_arbitrary__0.1_clf_.pickle'))
    model_pickle=os.path.join(syml_models_loc, model_name)

    pickle_to_file = pickle.load(open(model_pickle, "rb")) #load your model

    best_estimator = pickle_to_file['SVC'] #.best_estimator_
    print('Your symbol is:', symbol, 'and the model date is:' ,common_date)
    # set a few OOS dates
    
    fwd_dates_list = sorted([i for i in common_dates if i > first_oos_day])[:3]
    
    fitted_models_results = {
            'accuracy': defaultdict(dict),
            'recall': defaultdict(dict),
            'F1-score': defaultdict(dict)
    }
    oos_features_path= os.path.join(syml_features_loc, common_date)
    for dic_idx, fwd_date in enumerate(fwd_dates_list):
        feature_file = "_".join((symbol,'3_states_features_date:',fwd_date,'now:',symbol_compute_date,'.pickle'))
        features_loc = os.path.join(syml_features_loc,common_date, feature_file)
        features_tuple=pickle.load(open(features_loc, "rb"))
        market_data_oos= pd.read_csv(os.path.join(syml_labels_loc, 
                                                  '.'.join((fwd_date,'csv'))),index_col=0)
        features_df = pd.concat([features_tuple[0], features_tuple[1],
                             features_tuple[2], features_tuple[3]], axis=1)
        print features_df.head()
        df_w_market_features = MarketFeatures(df=MarketFeatures(\
                                                                df=MarketFeatures(
                        df=MarketFeatures(df=market_data_oos).obv_calc()).chaikin_mf()).ma_spread()).ma_spread_duration()
# fix the sort issue!!
        df_concat = pd.concat([features_df, df_w_market_features], axis=1).dropna()

        label_name = str(df_concat.columns[df_concat.columns.str.contains(pat='label')].values[0])

        df_final = df_concat.drop(columns=['TradedPrice', 'Duration', 'TradedTime', 'ReturnTradedPrice', \
                                           'Volume', label_name])
        if len(df_final)> 5:
            X = MinMaxScaler().fit_transform(df_final)

            y_labels = df_concat[df_concat.columns[df_concat.columns.str.contains(pat='label')]].iloc[:, 0]
            y_predict = best_estimator.predict(X)
            accuracy_models_results[common_date][datetime.strptime(fwd_date, '%Y%m%d').strftime('%Y%m%d')] =(accuracy_score(y_labels, y_predict))
            recall_models_results[common_date][datetime.strptime(fwd_date, '%Y%m%d').strftime('%Y%m%d')] =(recall_score(y_labels,y_predict))
            f1_models_results[common_date][datetime.strptime(fwd_date, '%Y%m%d').strftime('%Y%m%d')]=f1_score(y_true= y_labels, y_pred=y_predict)
        else:
            print ('skipping')
fitted_models_results['accuracy']= accuracy_models_results
fitted_models_results['recall']= recall_models_results
fitted_models_results['F1-score']= f1_models_results

   


MKS.L
('Your symbol is:', 'MKS.L', 'and the model date is:', '20170920')
   fischer_score_dlambda  fischer_score_dsigma  fischer_score_dweight  \
0           7.405062e-16              0.000000               1.458914   
1           5.521199e+00          -2987.397931              -0.068926   
2           2.133214e+00           4715.734276              -1.599811   
3           1.274399e+00           4269.353587              -3.126769   
4          -2.456737e+01          11327.433895              -4.819308   

   lambda_lambda  lambda_sigma  lambda_weight   sigma_sigma  sigma_weight  \
0      30.495539      0.000000      15.986455  0.000000e+00      0.000000   
1     -36.053276      8.395049      -0.005938  8.309495e+06     -0.309868   
2     -73.102044   -674.690811      -0.178269 -8.495978e+07     24.903357   
3    -108.833626   -378.936567      -0.137239 -1.008749e+08     13.986840   
4    -216.945516 -32542.947971      -8.469757 -1.820023e+08   1201.185243   

   weight_weight       ga

   fischer_score_dlambda  fischer_score_dsigma  fischer_score_dweight  \
0           1.196493e-10              0.000000               1.209812   
1          -1.224049e+01              0.000000               3.754327   
2          -1.042635e+01              0.000000               5.284224   
3          -7.449227e+00            -61.404913               3.421601   
4          -3.215029e+00            -57.098907               5.643002   

   lambda_lambda  lambda_sigma  lambda_weight   sigma_sigma  sigma_weight  \
0  -4.861833e-10      0.000000   1.447531e-10  0.000000e+00      0.000000   
1  -2.105994e+03      0.000000   2.782667e-04  0.000000e+00      0.000000   
2  -2.111106e+03      0.000000   1.543176e+00  0.000000e+00      0.000000   
3  -2.124494e+03   -704.979937   1.456215e+00 -2.295313e+07     -6.009476   
4  -2.146279e+03   -536.245374   1.834194e+00 -2.305759e+07     -4.560012   

   weight_weight       gamma_0      ...            gamma_2    ksi_0_to_0  \
0       1.463646  1.00

('Your symbol is:', 'MKS.L', 'and the model date is:', '20170926')
   fischer_score_dlambda  fischer_score_dsigma  fischer_score_dweight  \
0                    NaN                   NaN                    NaN   
1                    NaN                   NaN                    NaN   
2                    NaN                   NaN                    NaN   
3                    NaN                   NaN                    NaN   
4                    NaN                   NaN                    NaN   

   lambda_lambda  lambda_sigma  lambda_weight  sigma_sigma  sigma_weight  \
0            NaN           NaN            NaN          NaN           NaN   
1            NaN           NaN            NaN          NaN           NaN   
2            NaN           NaN            NaN          NaN           NaN   
3            NaN           NaN            NaN          NaN           NaN   
4            NaN           NaN            NaN          NaN           NaN   

   weight_weight  gamma_0     ...    

   fischer_score_dlambda  fischer_score_dsigma  fischer_score_dweight  \
0           3.567851e-16              0.000000               1.561301   
1           4.303242e+00          13746.550285              -0.242851   
2           6.379895e+00          14118.461573               2.178990   
3           8.559431e+00          16840.448282              -1.172741   
4           2.389205e+01          17099.905569              -4.443890   

   lambda_lambda  lambda_sigma  lambda_weight   sigma_sigma  sigma_weight  \
0       4.513872      0.000000       5.111997  0.000000e+00      0.000000   
1     -24.577219 -33077.164376      -3.459803 -1.524858e+08   1304.102138   
2     -15.699387 -22110.757402      -3.360827 -1.618187e+08    836.238473   
3   -1793.853495 -34001.070820      -7.192771 -1.728060e+08   1217.355979   
4   -3573.728763 -34119.844023      -7.200642 -1.773885e+08   1222.038762   

   weight_weight       gamma_0      ...             gamma_2    ksi_0_to_0  \
0       5.789379  6.9

   fischer_score_dlambda  fischer_score_dsigma  fischer_score_dweight  \
0           2.036781e-16              0.000000               2.258652   
1           1.941631e+01          11729.672870               0.516282   
2           3.079354e+01          10022.897214              -1.964095   
3           5.001486e+01          27077.445877              -3.725931   
4           6.850382e+01          26093.373192              -5.468301   

   lambda_lambda  lambda_sigma  lambda_weight   sigma_sigma  sigma_weight  \
0  -4.163671e-15  0.000000e+00   4.600380e-16  0.000000e+00  0.000000e+00   
1  -3.974494e+02  2.910383e-11   7.105427e-15 -8.661497e+07  1.818989e-12   
2  -5.382685e+02 -5.853423e+02   8.396499e+00 -8.853812e+07 -1.838458e+01   
3  -9.321569e+02 -5.370699e+02   8.503693e+00 -2.102505e+08 -2.135642e+01   
4  -1.329606e+03 -5.370699e+02   8.503693e+00 -2.133719e+08 -2.135642e+01   

   weight_weight        gamma_0      ...            gamma_2     ksi_0_to_0  \
0       5.101507   1

...

('Your symbol is:', 'MKS.L', 'and the model date is:', '20180409')
   fischer_score_dlambda  fischer_score_dsigma  fischer_score_dweight  \
0           3.263846e-16              0.000000               1.375370   
1           1.478631e+01           4715.071719              -0.492643   
2           1.707572e+01           4152.334322              -2.360655   
3           5.755820e+01           3595.631035              -6.702714   
4           5.814371e+01           3596.297045              -5.639680   

   lambda_lambda  lambda_sigma  lambda_weight   sigma_sigma  sigma_weight  \
0    2276.040558      0.000000     102.670050  0.000000e+00      0.000000   
1   -2323.748412     -0.000064       0.000003 -4.364932e+07     -0.000035   
2   -4647.496846     -0.000021       0.000001 -5.051940e+07     -0.000012   
3   -6557.981270   -308.382762     103.766826 -5.747499e+07   -169.699069   
4   -6528.907835   -329.465019     112.469736 -5.748358e+07   -185.175159   

   weight_weight       gamma_0 

skipping
   fischer_score_dlambda  fischer_score_dsigma  fischer_score_dweight  \
0                    NaN                   NaN                    NaN   
1                    NaN                   NaN                    NaN   
2                    NaN                   NaN                    NaN   
3                    NaN                   NaN                    NaN   
4                    NaN                   NaN                    NaN   

   lambda_lambda  lambda_sigma  lambda_weight  sigma_sigma  sigma_weight  \
0            NaN           NaN            NaN          NaN           NaN   
1            NaN           NaN            NaN          NaN           NaN   
2            NaN           NaN            NaN          NaN           NaN   
3            NaN           NaN            NaN          NaN           NaN   
4            NaN           NaN            NaN          NaN           NaN   

   weight_weight  gamma_0     ...      gamma_2  ksi_0_to_0  ksi_0_to_1  \
0            NaN     

('Your symbol is:', 'MKS.L', 'and the model date is:', '20180412')
   fischer_score_dlambda  fischer_score_dsigma  fischer_score_dweight  \
0                    NaN                   NaN                    NaN   
1                    NaN                   NaN                    NaN   
2                    NaN                   NaN                    NaN   
3                    NaN                   NaN                    NaN   
4                    NaN                   NaN                    NaN   

   lambda_lambda  lambda_sigma  lambda_weight  sigma_sigma  sigma_weight  \
0            NaN           NaN            NaN          NaN           NaN   
1            NaN           NaN            NaN          NaN           NaN   
2            NaN           NaN            NaN          NaN           NaN   
3            NaN           NaN            NaN          NaN           NaN   
4            NaN           NaN            NaN          NaN           NaN   

   weight_weight  gamma_0     ...    

   fischer_score_dlambda  fischer_score_dsigma  fischer_score_dweight  \
0                    NaN                   NaN                    NaN   
1                    NaN                   NaN                    NaN   
2                    NaN                   NaN                    NaN   
3                    NaN                   NaN                    NaN   
4                    NaN                   NaN                    NaN   

   lambda_lambda  lambda_sigma  lambda_weight  sigma_sigma  sigma_weight  \
0            NaN           NaN            NaN          NaN           NaN   
1            NaN           NaN            NaN          NaN           NaN   
2            NaN           NaN            NaN          NaN           NaN   
3            NaN           NaN            NaN          NaN           NaN   
4            NaN           NaN            NaN          NaN           NaN   

   weight_weight  gamma_0     ...      gamma_2  ksi_0_to_0  ksi_0_to_1  \
0            NaN      NaN     

skipping
('Your symbol is:', 'MKS.L', 'and the model date is:', '20180419')
('Your symbol is:', 'MKS.L', 'and the model date is:', '20180420')


In [15]:
fitted_models_results['accuracy']

defaultdict(dict,
            {'20170920': {'20170922': 0.8561038961038961,
              '20170925': 0.9394919168591224,
              '20170926': 0.859514687100894},
             '20170922': {'20170925': 0.03879907621247113,
              '20170926': 0.10344827586206896,
              '20170927': 0.07523245984784446},
             '20170925': {'20170927': 0.3905325443786982,
              '20170928': 0.21771009588268472,
              '20170929': 0.7878675102344622},
             '20170927': {'20170929': 0.8823967249720879,
              '20180201': 0.838276440962507,
              '20180202': 0.926796050391556},
             '20180201': {'20180205': 0.7732469512195121,
              '20180206': 0.8201046663759267,
              '20180207': 0.8515912897822445},
             '20180202': {'20180205': 0.12195121951219512,
              '20180206': 0.14849542084605322,
              '20180207': 0.08676716917922948},
             '20180212': {'20180214': 0.6943537097526832,
              

In [ ]:
features_df